Data Source: https://www.kaggle.com/datasets/mdwaquarazam/stock-price-history-top-10-companies

In [1]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go
import os
import re

In [2]:
os.listdir("data/Stock-History")

['TSLA.csv',
 'WL-MART.csv',
 'AMZN.csv',
 'MSFOT.csv',
 'ZAll_Combine_Stock_Histry.csv',
 'VISA.csv',
 'AAPL.csv',
 'GOOGL.csv',
 'NTFLX.csv',
 'JP-MORGN.csv',
 'NVDA.csv']

In [3]:
AAPL_df = pd.read_csv('data/Stock-History/'+"AAPL.csv")
AAPL_df

,Date,Open,High,Low,Close,Adj Close,Volume,Company
0,07-08-2017,39.264999,39.730000,39.167500,39.702499,37.585022,87481200,AAPL
1,08-08-2017,39.650002,40.457500,39.567501,40.020000,37.885582,144823600,AAPL
2,09-08-2017,39.814999,40.317501,39.777500,40.264999,38.117512,104526000,AAPL
3,10-08-2017,39.974998,40.000000,38.657501,38.830002,36.903397,163217200,AAPL
4,11-08-2017,39.150002,39.642502,39.017502,39.369999,37.416603,105028400,AAPL
...,...,...,...,...,...,...,...,...
1253,29-07-2022,161.240005,163.630005,159.500000,162.509995,162.509995,101689200,AAPL
1254,01-08-2022,161.009995,163.589996,160.889999,161.509995,161.509995,67829400,AAPL
1255,02-08-2022,160.100006,162.410004,159.630005,160.009995,160.009995,59907000,AAPL
1256,03-08-2022,160.839996,166.589996,160.750000,166.130005,166.130005,82507500,AAPL


In [4]:
stocks={}
for file in os.listdir("data/Stock-History"):
    oexten = os.path.splitext(file)[0]
    stocks[oexten] = pd.read_csv('data/Stock-History/'+file)
    stocks[oexten] = stocks[oexten].drop(columns = ['High', "Low", "Adj Close", "Company"] )
    stocks[oexten] = stocks[oexten].rename(columns={c: c+"_"+ oexten for c in stocks[oexten].columns if c != "Date"})

In [5]:
stocks['AAPL']

,Date,Open_AAPL,Close_AAPL,Volume_AAPL
0,07-08-2017,39.264999,39.702499,87481200
1,08-08-2017,39.650002,40.020000,144823600
2,09-08-2017,39.814999,40.264999,104526000
3,10-08-2017,39.974998,38.830002,163217200
4,11-08-2017,39.150002,39.369999,105028400
...,...,...,...,...
1253,29-07-2022,161.240005,162.509995,101689200
1254,01-08-2022,161.009995,161.509995,67829400
1255,02-08-2022,160.100006,160.009995,59907000
1256,03-08-2022,160.839996,166.130005,82507500


In [6]:
del stocks['ZAll_Combine_Stock_Histry']

In [7]:
stocks.keys()

dict_keys(['TSLA', 'WL-MART', 'AMZN', 'MSFOT', 'VISA', 'AAPL', 'GOOGL', 'NTFLX', 'JP-MORGN', 'NVDA'])

In [8]:
stock_df = pd.DataFrame()

In [9]:
stock_df["Date"] = stocks["TSLA"]["Date"]

In [10]:
for stock in stocks:
    stock_df[stock] = stocks[stock]["Open_"+stock]

In [11]:
stock_df

,Date,TSLA,WL-MART,AMZN,MSFOT,VISA,AAPL,GOOGL,NTFLX,JP-MORGN,NVDA
0,07-08-2017,71.470001,80.570000,49.532501,72.800003,100.919998,39.264999,46.452999,181.000000,93.889999,42.097500
1,08-08-2017,71.505997,81.169998,49.717499,72.089996,101.500000,39.650002,46.354500,181.369995,93.949997,43.472500
2,09-08-2017,72.199997,81.110001,49.130001,72.250000,100.760002,39.814999,46.030499,171.429993,92.980003,42.107498
3,10-08-2017,72.320000,81.070000,48.814999,71.900002,100.540001,39.974998,45.877499,174.029999,92.900002,43.040001
4,11-08-2017,71.393997,80.730003,48.000000,71.610001,99.550003,39.150002,45.398499,169.860001,92.129997,39.285000
...,...,...,...,...,...,...,...,...,...,...,...
1253,29-07-2022,842.099976,128.320007,134.899994,277.700012,212.000000,161.240005,113.400002,223.289993,115.589996,178.130005
1254,01-08-2022,903.830017,131.059998,134.960007,277.820007,208.449997,161.009995,115.529999,223.100006,114.500000,181.820007
1255,02-08-2022,882.010010,133.149994,134.720001,276.000000,207.800003,160.100006,114.430000,222.759995,113.919998,181.220001
1256,03-08-2022,915.000000,132.160004,136.210007,276.760010,207.990005,160.839996,116.339996,224.789993,113.449997,181.839996


In [13]:
stock_df.to_csv("stock.csv")